## Cosine similarity

In [2]:
import numpy as np   

def cosine(v1, v2):
    v1, v2 = np.array(v1), np.array(v2)
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

## Load embeddings
https://nlp.stanford.edu/projects/glove/

In [3]:
def load_embeddings(path):
    mapping = dict()
    
    with open(path, 'r', encoding='utf8') as f:
        for line in f:
            line = line.strip()
            if len(line) == 0:
                continue
            splitted = line.split(" ")
            mapping[splitted[0]] = np.array(splitted[1:], dtype=float)
    return mapping

mapping = load_embeddings('..\data\external\glove.6B.50d.txt')

<>:13: SyntaxWarning: invalid escape sequence '\d'
<>:13: SyntaxWarning: invalid escape sequence '\d'
C:\Users\User\AppData\Local\Temp\ipykernel_9748\2635219672.py:13: SyntaxWarning: invalid escape sequence '\d'
  mapping = load_embeddings('..\data\external\glove.6B.50d.txt')


## Text to embeddings
Average vector of word vectors in a text.

In [13]:
import nltk
nltk.download('punkt') #https://www.nltk.org/data.html

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [25]:
from nltk.tokenize import word_tokenize

def document_to_avg_embedding(doc, embeddings):
    
    doc_embeddings = []
    doc = str.lower(doc)
    tokens = word_tokenize(doc)

    for token in tokens:
        if token in embeddings.keys():
            doc_embeddings.append(embeddings[token])
    
    avg_embedding = np.mean(np.array(doc_embeddings), axis=0)

    return avg_embedding

## Cosine similarity between docs

In [28]:
def cosine_sim_between_docs(doc1, doc2, embeddings, verbose=False):

    v1 = document_to_avg_embedding(doc1, embeddings)
    v2 = document_to_avg_embedding(doc2, embeddings)
    cosine_sim = cosine(v1, v2)

    if verbose:
        print(f'Cosine similarity between {doc1} and {doc2} is:\n{cosine_sim}\n')

    return cosine_sim

Below results are not promising, however I must to test it on bigger dataset.

In [ ]:
cosine_sim_between_docs("Mary had a little lamb", "A girl had a small sheep", mapping, verbose=True) #0.8991286073011774
cosine_sim_between_docs("Mary had a little lamb", "This old man, he played one", mapping, verbose=True) #0.906964976899118
cosine_sim_between_docs("Mary had a little lamb", "Twinkle Twinkle Little Star How I wonder what you are!", mapping, verbose=True) #0.7681287321680009
cosine_sim_between_docs("Mary had a little lamb", "Your STIHL earth auger uses an oil-gasoline mixture for fuel (see the chapter on Fuel in your owner's manual).", mapping, verbose=True) #0.8210465662463234

Cosine similarity between Mary had a little lamb and A girl had a small sheep is:
0.8991286073011774

Cosine similarity between Mary had a little lamb and This old man, he played one is:
0.906964976899118

Cosine similarity between Mary had a little lamb and Twinkle Twinkle Little Star How I wonder what you are! is:
0.7681287321680009

Cosine similarity between Mary had a little lamb and Your STIHL earth auger uses an oil-gasoline mixture for fuel (see the chapter on Fuel in your owner's manual). is:
0.8210465662463234



0.8210465662463234